In [294]:
import json
from fastapi import FastAPI
import gensim

import scipy

import sys
sys.path.append('./models')

from fastapi.middleware.cors import CORSMiddleware

import pickle 
import os
import glob

In [2]:
from rank_bm25 import BM25Okapi
from pymongo import MongoClient
import psycopg2

# Connect to the PostgreSQL database
conn = psycopg2.connect(
    host="localhost",
    database="SearchEngine",
    user="postgres",
    password="2580"
)

# Create a cursor object
cursor = conn.cursor()


In [3]:
app = FastAPI()

origins = [
    "http://localhost.tiangolo.com",
    "https://localhost.tiangolo.com",
    "http://localhost",
    "http://localhost:8080",
]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

In [65]:
user_input = 'London Rishi'

In [66]:
client = MongoClient('mongodb://localhost:27017/')
# select a database
db = client['inverted_index']
mycollection = db['inverted_index']

In [255]:
words = user_input.split()

In [256]:
words

['London', 'Rishi']

In [267]:
words = user_input.split()

rel_docs = set()
for word in words:
    myquery = {"word": word}
    results  = mycollection.find(myquery)
    for result in results:
        docs = result['doc_ids']
        print(word,docs)
        rel_docs.update(docs)

London [10, 23, 43, 56, 102, 110, 117, 124, 180, 183, 212, 215, 217, 235, 257, 283, 286, 290, 291, 346, 357, 380, 388, 395, 397, 404, 407, 422, 424, 443, 466, 475, 482, 501, 515, 519, 523, 528, 530, 541, 553, 606, 613, 620, 680, 710, 713, 715, 734, 755, 781, 784, 787, 788, 844, 855, 877, 892, 894, 900, 903, 906, 917, 921, 923, 965, 980, 998, 1013, 1018, 1026, 1027, 1038, 1050, 1096, 1104, 1111, 1119, 1175, 1178, 1208, 1211, 1213, 1231, 1253, 1279, 1282, 1285, 1286, 1315, 1348, 1382, 1402, 1420, 1423, 1473, 1479, 1513, 1519, 1551, 1641, 1741, 1746, 1829, 1907]
Rishi [5, 39, 81, 146, 192, 267, 354, 381, 490, 506, 508, 526, 536, 578, 642, 689, 852, 857, 878, 988, 1003, 1005, 1024, 1033, 1075, 1141, 1188, 1263, 1309, 1310, 1311, 1315, 1317, 1322, 1324, 1325, 1329, 1332, 1335, 1338, 1339, 1341, 1342, 1346, 1348, 1349, 1350, 1355, 1356, 1359, 1365, 1367, 1368, 1371, 1372, 1373, 1381, 1386, 1389, 1392, 1395, 1396, 1397, 1398, 1401, 1402, 1403, 1404, 1405, 1411, 1412, 1413, 1417, 1419, 1423, 1

In [269]:
len(rel_docs)

325

In [371]:
def reated_documents(user_input):
    words = user_input.split()
    rel_docs = set()
    for word in words:
        myquery = {"word": word}
        results  = mycollection.find(myquery)
        for result in results:
            docs = result['doc_ids']
            rel_docs.update(docs)
            
    return list(rel_docs)

In [273]:
len(reated_documents('London Rishi'))

['London', 'Rishi']
London
Rishi


325

In [265]:
rel_docs = list(rel_docs)

In [266]:
len(rel_docs)

325

In [72]:
documents  = []
urls = []
for idx in rel_docs:
    query = f"SELECT * FROM sites where id = {idx};"
    cursor.execute(query)
    results = cursor.fetchall()
    for row in results:
        doc_id = row[0]
        url = row[1]
        urls.append(url)
        text = row[2]
        documents.append(text)

In [73]:
tokenized_corpus = [doc.split(" ") for doc in documents]
bm25 = BM25Okapi(tokenized_corpus)


In [74]:
tokenized_query = words
doc_scores = bm25.get_scores(tokenized_query)
doc_scores = doc_scores.argsort()[::-1]

In [75]:
doc_scores

array([226,  80, 269, 134, 125, 293, 187,  27,  18,  73,  93, 262, 273,
        67, 132, 206, 194, 176, 292, 265, 275, 189, 100, 317, 270,   5,
       110, 111, 106, 112, 109, 108,  87,  86,  85, 107,  94,  88,  95,
       104, 103,  89, 102,  90, 101, 113,  99,  91,  98,  97,  92,  96,
       105, 324, 114, 144, 137, 138, 139, 140, 141, 142, 143, 145, 115,
       146, 147, 148, 149, 150, 151, 152, 136, 135, 133, 131, 116, 117,
        83, 118, 119, 120, 121, 122, 123, 124, 126, 127, 128, 129, 130,
        84,  75,  82,  30,  22,  23,  24,  25,  26,  28,  29,  31,  40,
        32,  33,  34,  35,  36,  37,  38,  21,  20,  19,  17,   1,   2,
         3,   4,   6,   7,   8,   9,  10,  11,  12,  13,  14,  15,  16,
        39,  41,  81,  70,  62,  63,  64,  65,  66,  68,  69,  71,  42,
        72,  74, 154,  76,  77,  78,  79,  61,  60,  59,  58,  43,  44,
        45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,
       153, 162, 155, 268, 259, 260, 261, 263, 264, 266, 267, 27

In [76]:
rel_docs[2]

23

In [57]:
out_data = {}
for idx,score_id in enumerate(doc_scores,start=1):
    doc_id = rel_docs[score_id]
    url = urls[score_id]
    text = documents[score_id][:200]
    out_data.update({idx:{'rank':idx,'doc_id':doc_id,'url':url,'text':text }})

In [58]:
len(out_data)

3

In [59]:
json_data = json.dumps(out_data)

In [60]:
json_data

'{"1": {"rank": 1, "doc_id": 630, "url": "https://www.bbc.com/sport/rugby-union/64760101", "text": "Premiership: Exeter Chiefs 24-22 Sale Sharks- Josh Hodge double helps home side to victory  Last updated on 26 February 202326 February 2023.From the section Rugby Union Josh Hodge\'s two first-half tr"}, "2": {"rank": 2, "doc_id": 133, "url": "https://www.bbc.com/news/england/lincolnshire", "text": "BBC News Lincolnshire  Sir Edward Leigh says RAF Scampton could be used as part of efforts to move migrants from hotels. Emma Parker is jailed for 12 months for the \\"abhorrent\\" cruelty filmed at her h"}, "3": {"rank": 3, "doc_id": 1128, "url": "https://www.bbc.com/sport/articles/c04p0r0ly8lo", "text": "Dyche on scoring goals, away form and Arsenal Joe Bradshaw, BBC Sport Sean Dyche has been speaking to the media before Everton\\u00e2\\u0080\\u0099s game at Arsenal on Wednesday. Here are the key lines from his news co"}}'

In [62]:
doc_id = rel_docs[2]

In [63]:
doc_id

630

In [64]:
query = f"SELECT * FROM sites where id = {doc_id};"
cursor.execute(query)
results = cursor.fetchall()
for row in results:
    print(row)

(630, 'https://www.bbc.com/sport/rugby-union/64760101', 'Premiership: Exeter Chiefs 24-22 Sale Sharks- Josh Hodge double helps home side to victory  Last updated on 26 February 202326 February 2023.From the section Rugby Union Josh Hodge\'s two first-half tries helped Exeter beat Sale in a thrilling Premiership encounter at Sandy Park. Tom Roebuck\'s outstanding solo try put Sale 10-0 up, but Harry Williams and Hodge\'s two excellent scores with Sharks down to 13 men put Exeter ahead.  Ewan Ashman\'s try cut the gap as Sale started the second half well.  They continued to up the pressure and took the lead with 13 minutes to go through Sam Dugdale, before Harvey Skinner\'s penalty won it for Exeter. The victory moves Exeter up to fifth place in the Premiership, a point off fourth-placed Gloucester with a game in hand, while Sale remain second and 11 points behind Saracens after earning a losing bonus point.  Having weathered an early wave of Exeter pressure, Sale began to assert themsel

## TESTING WORD2VWC

As mentioned in the DESM paper, we compute the centroid of documents and compute the cosine similarity of the query words with document centroid.

In [221]:
query = f"SELECT content FROM sites;"
cursor.execute(query)
results = cursor.fetchall()


In [222]:
# (results)

In [19]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('gutenberg')
import string
from nltk.corpus import stopwords
from nltk import word_tokenize
from gensim.models import Word2Vec as w2v


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\iq\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\iq\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\iq\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [20]:
sw = stopwords.words('English') # this is a list of stopwords
# Function to remove stopwords per line if they are present in the line
def rem_stops_line(line, words):
    if len(line) >1:
        return [w for w in line if w not in words]
    else:
        return line

# Remove stop words for an entire text. Separate functions make it easier to parallelize if required.
def remove_stops(text, words = sw):
    return [rem_stops_line(line, words) for line in text]

In [21]:
lines = [line[0].rstrip('\n').lower() for line in results]

In [22]:
lines = [line.translate(str.maketrans('', '', string.punctuation)) for line in lines]
lines = [word_tokenize(line) for line in lines]
filtered_lines = remove_stops(text = lines, words = sw)

In [ ]:
words = set()
for line in filtered_lines:
    for word in line:
        words.add(word)
len(words)

In [186]:
model1 = w2v(
    filtered_lines,
    min_count=3,
    sg = 1,       # 1 for skip-gram, 0 for cbow
    window=7   # sliding window size
)


In [187]:
# print(w.wv.most_similar('nhs'))
# 

In [188]:
model1.save('./models/w2v-lc.model')
model1.wv.save_word2vec_format('./models/w2v-lc.model.bin', binary=True)

vocab =  model1.wv.key_to_index

with open('./models/w2v-lc-vocab.json', 'w') as f:
    f.write(json.dumps(vocab))

In [190]:
model1.wv.most_similar(positive=['nhs', 'rishi'], negative=['alabama'])

[('sunak', 0.5008801817893982),
 ('ministers', 0.45453718304634094),
 ('sunaks', 0.43160659074783325),
 ('keir', 0.42182454466819763),
 ('prime', 0.4193575978279114),
 ('deal', 0.41837599873542786),
 ('stability', 0.41807878017425537),
 ('secretary', 0.41506227850914),
 ('minister', 0.4142037630081177),
 ('brexit', 0.4126107096672058)]

In [185]:
len(model1.wv.key_to_index.keys())

54339

In [138]:
import numpy as np

def get_embedding(x, out=False):
    if x in model1.wv.key_to_index:
        if out:
            return model1.syn1neg[model1.wv.key_to_index[x]]
        else:
            return model1.wv[x]
    else:
        return np.zeros(100)


In [229]:
query = f"SELECT id,content FROM sites;"
cursor.execute(query)
results = cursor.fetchall()

In [241]:

for document in results:
    line = document[1]
    id_ = document[0]
    fname = f'centroid_file_{id_}'
    
    centroid_in = (np.mean(np.array([get_embedding(x) for x in nltk.word_tokenize(line.lower())]), axis=0))
    centroid_out = (np.mean(np.array([get_embedding(x, out=True) for x in nltk.word_tokenize(line.lower())]), axis=0))
    
    out_dict = { fname : (centroid_in, centroid_out) }
    pickle_file = './inputs/centroids/' + os.path.basename(fname).replace('.txt', '.p')
    pickle.dump(out_dict, open(pickle_file, "wb"))
    

In [215]:
model1 = gensim.models.Word2Vec.load('./models/w2v-lc.model')

In [361]:
def score_document(q_embeddings, doc_id,scope='in'):
    fname = f'centroid_file_{doc_id}'    
    centroid_dict = {}
    centroid_dict.update(pickle.load(open(f'./inputs/centroids/{fname}', "rb")))
    clean_centroid_dict = {k: centroid_dict[k] for k in centroid_dict if not np.isnan(centroid_dict[k][0]).any()}
    
    centroid_dict = clean_centroid_dict[fname]
    
    if scope=='in':
        centroid = centroid_dict[0]
        individual_csims = [(1 - scipy.spatial.distance.cosine(qin, centroid)) for qin in q_embeddings]
    else:
        centroid = centroid_dict[1]
        individual_csims = [(1 - scipy.spatial.distance.cosine(qin, centroid)) for qin in q_embeddings]
    return (sum(individual_csims)/len(q_embeddings))

In [368]:
def DESM_Scoring(user_query):
    query_words = user_query.split()
    rel_docs = reated_documents(user_query)
    q_embeddings = [get_embedding(x.lower()) for x in query_words]
    DESM_scores = [score_document(q_embeddings,doc) for doc in rel_docs]
    score_docs = list(zip(rel_docs,DESM_scores))
    
    return score_docs

In [370]:
DESM_Scoring('Rishi London')

['Rishi', 'London']
Rishi
London


[(5, 0.4073741912979941),
 (10, 0.4417821233070274),
 (23, 0.45805694403666725),
 (39, 0.33824270963668823),
 (43, 0.4650102641302769),
 (56, 0.48007644214226686),
 (81, 0.4370575159885602),
 (102, 0.3866954495035115),
 (110, 0.4372519301070651),
 (117, 0.44276347408459205),
 (124, 0.4722912475676667),
 (146, 0.4582764506819532),
 (180, 0.37392967748723605),
 (183, 0.28097719229486023),
 (192, 0.38228567252348694),
 (212, 0.3609492194549513),
 (215, 0.4210878932109272),
 (217, 0.4451685603711271),
 (235, 0.5204974793772789),
 (257, 0.48340432289430396),
 (267, 0.4213556622272215),
 (283, 0.38965407887905934),
 (286, 0.39142724486266584),
 (290, 0.4419164353763412),
 (291, 0.41838211251598656),
 (346, 0.3959855727601687),
 (354, 0.4323780866299338),
 (357, 0.5296072234150817),
 (380, 0.42020773248077253),
 (381, 0.3952583530330009),
 (388, 0.418638046617318),
 (395, 0.4017533433849373),
 (397, 0.38613208204607036),
 (404, 0.3797786905477742),
 (407, 0.3727653993608281),
 (422, 0.4456689

In [365]:
DESM_scores = [score_document(q_embeddings,doc) for doc in rel_docs]

In [367]:
score_docs = list(zip(rel_docs,DESM_scores))

In [352]:
score_document(q_embeddings,23)

0.4020382247129924

In [385]:
from models.desm import DESM_Scoring

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\iq\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\iq\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\iq\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [386]:
DESM_Scoring('Rishi London')

[(5, 0.4295209302159811),
 (10, 0.4494684305894696),
 (23, 0.46757226663396295),
 (39, 0.35704030096530914),
 (43, 0.47898931096371083),
 (56, 0.4894335911287991),
 (81, 0.4601226422554214),
 (102, 0.37346664338216706),
 (110, 0.44587018692556946),
 (117, 0.4511715732104508),
 (124, 0.4900499839620152),
 (146, 0.47333582644356376),
 (180, 0.4037717302315668),
 (183, 0.28897264971268044),
 (192, 0.3934593088271918),
 (212, 0.38521712587857093),
 (215, 0.4336355269879981),
 (217, 0.46227504043223566),
 (235, 0.540018284813073),
 (257, 0.5091735163939375),
 (267, 0.4384569802104027),
 (283, 0.40752111907042965),
 (286, 0.41577397814279854),
 (290, 0.4500093834004313),
 (291, 0.4394703223525878),
 (346, 0.40270892609042075),
 (354, 0.4482857675452184),
 (357, 0.5438157918094071),
 (380, 0.44338655799253934),
 (381, 0.4194127958527299),
 (388, 0.43637403848921846),
 (395, 0.41603584883690425),
 (397, 0.4134905827929204),
 (404, 0.4075633807806093),
 (407, 0.3915073686268118),
 (422, 0.45741